In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
np.seterr(over='raise', invalid='raise', divide='raise');

## Load the training data into feature matrix, class labels, and event ids:

In [ ]:
from proj1_helpers import *
from helpers import *
DATA_TRAIN_PATH = '../train.csv' # TODO: download train data and supply path here 
y, X, ids = load_csv_data(DATA_TRAIN_PATH)

In [ ]:
from preprocessing import *
X0, X1, X2, indices0, indices1, indices2 = separate_data(X)
y0 = y[indices0]
y1 = y[indices1]
y2 = y[indices2]
overall_acc = []

In [ ]:
X2[0,29]

In [ ]:
from toolbox import *
# Define the parameters of the algorithm.
max_iters0 = 13000
gamma0 = 1e-6
lambda_0 = 0
degree_0 = 2
batch_size0 = 100
k = 4
current_y = y0
current_X = X0
k_indices = build_k_indices(current_y, k, 1)

loss_tr = []
loss_te = []
classification_acc = []
for k_ in range(k):

    test_indices = k_indices[k_]
    test_y, test_x = (current_y[test_indices], current_X[test_indices])
    test_x = build_poly_cos_sin_poly(test_x, degree_0, np.array(range(1,19)))

    training_indices = np.ravel(np.delete(k_indices, k_, axis=0))
    training_y, training_x = (current_y[training_indices], current_X[training_indices])
    training_x = build_poly_cos_sin_poly(training_x, degree_0, np.array(range(1,19)))

    w_star = reg_logistic_regression(training_y, training_x, lambda_0, gamma0, max_iters0)
    
    # compute classification accuracy
    y_pred = predict_labels_log_regression(w_star, test_x)
    
    test_y[test_y == 0] = -1
    classification_acc.append(np.mean(np.abs(test_y + y_pred) / 2))
    test_y[test_y == -1] = 0

    loss_tr.append(calculate_loss_log_likelihood(training_y, training_x, w_star))
    loss_te.append(calculate_loss_log_likelihood(test_y, test_x, w_star))
        
print("Training error: {tr}\nTest error: {te}\nClassification accuracy: {cl}".format(tr=np.mean(loss_tr), te=np.mean(loss_te), cl=np.mean(classification_acc)))
overall_acc.append(np.mean(classification_acc))

In [ ]:
from toolbox import *
# Define the parameters of the algorithm.
max_iters_1 = 25000
gamma_1 = 0.9e-7
lambda_1 = 0
degree_1 = 3
batch_size1 = 100
k = 4
current_y = y1
current_X = X1
k_indices = build_k_indices(current_y, k, 1)

loss_tr = []
loss_te = []
classification_acc = []
for k_ in range(k):

    test_indices = k_indices[k_]
    test_y, test_x = (current_y[test_indices], current_X[test_indices])
    test_x = build_poly_cos_sin_poly(test_x, degree_1, np.array(range(1,23)))

    training_indices = np.ravel(np.delete(k_indices, k_, axis=0))
    training_y, training_x = (current_y[training_indices], current_X[training_indices])
    training_x = build_poly_cos_sin_poly(training_x, degree_1, np.array(range(1,23)))

    w_star = reg_logistic_regression(training_y, training_x, lambda_1, gamma_1, max_iters_1)
    
    # compute classification accuracy
    y_pred = predict_labels_log_regression(w_star, test_x)
    
    test_y[test_y == 0] = -1
    classification_acc.append(np.mean(np.abs(test_y + y_pred) / 2))
    test_y[test_y == -1] = 0

    loss_tr.append(calculate_loss_log_likelihood(training_y, training_x, w_star))
    loss_te.append(calculate_loss_log_likelihood(test_y, test_x, w_star))
        
print("Training error: {tr}\nTest error: {te}\nClassification accuracy: {cl}".format(tr=np.mean(loss_tr), te=np.mean(loss_te), cl=np.mean(classification_acc)))
overall_acc.append(np.mean(classification_acc))

In [ ]:
from toolbox import *
# Define the parameters of the algorithm.
max_iters_2 = 15000
gamma_2 = 1e-7
lambda_2 = 0.0
degree_2 = 3
batch_size2 = 100
k = 4
current_y = y2
current_X = X2
k_indices = build_k_indices(current_y, k, 1)

loss_tr = []
loss_te = []
classification_acc = []
for k_ in range(k):

    test_indices = k_indices[k_]
    test_y, test_x = (current_y[test_indices], current_X[test_indices])
    test_x = build_poly_cos_sin_poly(test_x, degree_2, np.array(range(1,50)))
    

    training_indices = np.ravel(np.delete(k_indices, k_, axis=0))
    training_y, training_x = (current_y[training_indices], current_X[training_indices])
    training_x = build_poly_cos_sin_poly(training_x, degree_2, np.array(range(1,50)))
    
    w_star = reg_logistic_regression(training_y, training_x, lambda_2, gamma_2, max_iters_2)
    
    # compute classification accuracy
    y_pred = predict_labels_log_regression(w_star, test_x)
    
    test_y[test_y == 0] = -1
    classification_acc.append(np.mean(np.abs(test_y + y_pred) / 2))
    test_y[test_y == -1] = 0

    loss_tr.append(calculate_loss_log_likelihood(training_y, training_x, w_star))
    loss_te.append(calculate_loss_log_likelihood(test_y, test_x, w_star))
        
print("Training error: {tr}\nTest error: {te}\nClassification accuracy: {cl}".format(tr=np.mean(loss_tr), te=np.mean(loss_te), cl=np.mean(classification_acc)))
overall_acc.append(np.mean(classification_acc))

In [ ]:
print("The overall accuracy: {acc}\n".format(acc=np.mean(overall_acc)))

In [ ]:
from preprocessing import *

gamma_0 = 1e-6
max_iters_0 = 15000
degree_0 = 2

max_iters_1 = 25000
gamma_1 = 0.9e-7
degree_1 = 3

max_iters_2 = 15000
gamma_2 = 1e-7
degree_2 = 3

max_iters_3 = 15000
gamma_3 = 1e-6
degree_3 = 3

X0, X1, X2, X3, indices0, indices1, indices2, indices3 = separate_data(X)
y0 = y[indices0]
y1 = y[indices1]
y2 = y[indices2]
#y3 = y[indices3]


phi_X0 = build_poly_cos_sin_poly(X0, degree_0, np.array(range(1,19)))
phi_X1 = build_poly_cos_sin_poly(X1, degree_1, np.array(range(1,23)))
phi_X2 = build_poly_cos_sin_poly(X2, degree_2, np.array(range(1,30)))
#phi_X3 = build_poly(X3, degree_3)

w0 = reg_logistic_regression(y0, phi_X0, 0, gamma_0, max_iters_0)
w1 = reg_logistic_regression(y1, phi_X1, 0, gamma_1, max_iters_1)
w2 = reg_logistic_regression(y2, phi_X2, 0, gamma_2, max_iters_2)
#w3 = reg_logistic_regression(y3, phi_X3, 0, gamma_3, max_iters_3)

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = 'test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
X0_te, X1_te, X2_te, indices0_te, indices1_te, indices2_te = separate_data(tX_test)

In [ ]:
OUTPUT_PATH = '../submission6.csv' # TODO: fill in desired name of output file for submission
ids_test0 = ids_test[indices0_te]
ids_test1 = ids_test[indices1_te]
ids_test2 = ids_test[indices2_te]
#ids_test3 = ids_test[indices3_te]

phi_X0_te = build_poly_cos_sin_poly(X0_te, degree_0, np.array(range(1,19)))
phi_X1_te = build_poly_cos_sin_poly(X1_te, degree_1, np.array(range(1,23)))
phi_X2_te = build_poly_cos_sin_poly(X2_te, degree_2, np.array(range(1,30)))
#phi_X3_te = build_poly(X3_te, degree_3)

pred0 = predict_labels_log_regression(w0, phi_X0_te)
pred1 = predict_labels_log_regression(w1, phi_X1_te)
pred2 = predict_labels_log_regression(w2, phi_X2_te)
#pred3 = predict_labels_log_regression(w3, phi_X3_te)

predictions = np.append(pred0, np.append(pred1, pred2))
ids = np.append(ids_test0, np.append(ids_test1, ids_test2))

create_csv_submission(ids, predictions, OUTPUT_PATH)
